<a href="https://colab.research.google.com/github/ilucas75/BTflow/blob/dev/RD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install

In [1]:
!pip install pdfminer.six
!pip install spacy
!pip install tabula-py
!python -m spacy download fr_core_news_md

     |████████████████████████████████| 5.6MB 4.8MB/s 
     |████████████████████████████████| 13.7MB 241kB/s 
     |████████████████████████████████| 10.4MB 4.8MB/s 
     |████████████████████████████████| 88.6MB 1.3MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp36-none-any.whl size=90338488 sha256=522a3b5984b49df1ce935c5a4b88409bb7759be5b4961012542e4fe5b03b884d
  Stored in directory: /tmp/pip-ephem-wheel-cache-58bpqzcu/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


## Upload PDF and Json

In [35]:
from google.colab import files

uploaded = files.upload()
all_files = ' '.join(list(uploaded.keys()))
!mkdir -p data
!mv {all_files} ./data/

Saving 0070C022_220C0700.json to 0070C022_220C0700.json
Saving 0340C022_220C0430.json to 0340C022_220C0430.json
Saving 0510C022_220C0150.json to 0510C022_220C0150.json
Saving 0540C022_220C0450.json to 0540C022_220C0450.json
Saving 0620C022_220C0260.json to 0620C022_220C0260.json
Saving 0810C022_220C0180.json to 0810C022_220C0180.json
Saving 0820C022_220C0280.json to 0820C022_220C0280.json
Saving 0900C022_220C0090.json to 0900C022_220C0090 (2).json
Saving 0940C022_220C0490.json to 0940C022_220C0490 (2).json
Saving 0960C022_220C0690.json to 0960C022_220C0690 (2).json
Saving 0980C022_220C0890.json to 0980C022_220C0890 (2).json
Saving 1040C022_220C0401.json to 1040C022_220C0401 (2).json
Saving 1080C022_220C0801.json to 1080C022_220C0801 (2).json
Saving 1090C022_220C0901.json to 1090C022_220C0901 (2).json
Saving 1200C022_220C0021.json to 1200C022_220C0021 (2).json
Saving 1270C022_220C0721.json to 1270C022_220C0721 (2).json
Saving 1360C022_220C0631.json to 1360C022_220C0631 (2).json
Saving 1

## Table PDF

In [0]:
from tabula import read_pdf
import pandas as pd

def extract_tables(filename):
  dfs = read_pdf(list_pdf[0], pages='all')
  tables_str = []
  for df in dfs:
  if not df.empty:
    table_str = ""
    for c in df.columns:
      if not "Unnamed:" in c:
        table_str += c + " "
    for _, r in df.iterrows():
      for c in df.columns:
        if pd.notna(r[c]):
          table_str += r[c] + " "
    tables_str.append(table_str)
  return dfs, tables_str

Got stderr: Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Mar 31, 2020 7:14:33 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



## Extract PDF text

In [0]:
import io
 
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [0]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
 
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
 
        text = fake_file_handle.getvalue()
 
    # close open handles
    converter.close()
    fake_file_handle.close()
 
    if text:
        return text

## Load PDF and Json

In [37]:
import json
import os 

data = {}
data_label = {}
data_no_label = {}

for f in os.listdir('./data'):
  try:
    f = os.path.join('./data', f)
    name, ext = os.path.splitext(f)
    if ext == '.pdf' and name not in pdfs:
      text = extract_text_from_pdf(f)
      if "Déclaration de franchissement de seuils" in text:
        x =  {"file": f, "text": text}
        data[name] = x
        data_no_label[name] = x
    elif ext == '.json':
      with open(f) as fd:
        x = json.load(fd)
        data[name] = x
        if 'entities' in x and len(x['entities']) != 0:
          data_label[name] = x
          if name in data_no_label:
            del data_no_label[name]
        else:
          data_no_label[name] = x
  except Exception as e:
    print("Error : ", f)
    print(e)
data = [v for k, v in sorted(data.items(), key=lambda t: t[0])]
data_label = [v for k, v in sorted(data_label.items(), key=lambda t: t[0])]
data_no_label = [v for k, v in sorted(data_no_label.items(), key=lambda t: t[0])]
print(len(data), "texts with", len(data_label), "labeled and", len(data_no_label), 'to label')

117 texts with 117 labeled and 0 to label


## Labellisation

In [0]:
import IPython
from google.colab import output
import os

class AnnotWidget():
  def __init__(self, to_annot, start=0, model=None):
    self.to_annot = to_annot
    self.cur = start
    self.model = model
    self.colors_labels = [
        ("rgb(255, 0, 0)", "ISSUER", 'A'),
        ("rgb(255, 0, 255)", "FIRM", 'Z'),
        ("rgb(0, 255, 255)", "NUMBER", 'E'),
        ("rgb(160, 160, 255)", "PERCENT", 'R'),
        ("rgb(192, 192, 192)", "TYPE", 'T'),
        ("rgb(255, 255, 0)", "DIRECTION", 'S'),
        ("rgb(0, 255, 0)", "DATE", 'D'),
        ("rgb(255, 180, 180)", "FILIALE", 'F'),
        ("rgb(230, 171, 0)", 'INSTRUMENT', 'G')
      ]
    self.controls = ""
    for c, l, k in self.colors_labels:
      self.controls += '''<button id="key_'''+k+'''" style="background-color: '''+\
        c+''';" onclick='document.execCommand("backColor", false, "'''+c+'''")'>'''+l+" ("+k+")</button>"

  def text_entities_to_html(self):
    if 'entities' not in self.to_annot[self.cur] or len(self.to_annot[self.cur]['entities']) == 0:
      return self.to_annot[self.cur]["text"]
    res = ""
    offset = 0
    text = self.to_annot[self.cur]["text"]
    for ent_s, ent_e, label in self.to_annot[self.cur]['entities']:
      res += text[offset:ent_s]
      res += '''<span style="background-color: '''+ self.get_color(label) + '''">'''
      res += text[ent_s:ent_e]
      res += "</span>"
      offset = ent_e
    res += text[offset:]
    return res

  def run(self):
    display(IPython.display.HTML('''
    <p>'''+str(self.cur) + "/" + str(len(self.to_annot)) + '''</p>
    <fieldset>
    '''+ self.controls +'''
      <button id="key_Q" onclick="document.execCommand('removeFormat', false, null)">remove (Q)</button>
      <button id="key_P" onclick="google.colab.kernel.invokeFunction('notebook.Prediction');">pred (P)</button>
      <button id='prev'>prev</button>
      <button id='next'>next</button>
    </fieldset>
    <div id="ed" contenteditable="true">'''+self.text_entities_to_html()+'''</div>
    <script>
      function change(dir) {
        children = document.querySelector("#ed").childNodes;
        res = [];
        offset = 0;
        for (var i = 0; i < children.length; i++) {
          next_offset = offset + children[i].textContent.length;
          if (children[i].nodeType != Node.TEXT_NODE) {
            rgb = children[i].style.backgroundColor
            res.push([offset, next_offset, rgb])
          }
          offset = next_offset;
        }
        google.colab.kernel.invokeFunction('notebook.SaveEntities', [res, dir]);
        document.querySelector('#ed').innerHTML = next_text;
      };
      document.querySelector('#prev').onclick = () => {
        change(-1);
      };
      document.querySelector('#next').onclick = () => {
        change(1);
      };
      document.querySelector('#ed').onkeydown = (ev) => {
        ev.preventDefault();
      };
      document.querySelector('#ed').onkeypress = (ev) => {
        ev.preventDefault();
      };
      document.querySelector('#ed').onkeyup = (ev) => {
        ev.preventDefault();
        button_id = "#key_" + String.fromCharCode(ev.keyCode);
        button = document.querySelector(button_id)
        if (button) {
          button.click();
        }
      };
    </script>
    '''))
    output.register_callback('notebook.SaveEntities', self.save_entities)
    output.register_callback('notebook.Console', self.console)
    output.register_callback('notebook.Prediction', self.prediction)

  def get_color(self, label):
    for c, l, _ in self.colors_labels:
        if l == label:
          return c

  def get_label(self, color):
    for c, l, _ in self.colors_labels:
        if c == color:
          return l

  def save_entities(self, res, inc=1):
    self.to_annot[self.cur]['entities'] = []
    for ent_s, ent_e, color in res:
      self.to_annot[self.cur]['entities'].append((ent_s, ent_e, self.get_label(color)))
    if len(self.to_annot[self.cur]['entities']) != 0:
      name, _ = os.path.splitext(self.to_annot[self.cur]['file'])
      with open('./data/'+name+'.json', 'w+') as f:
        json.dump(self.to_annot[self.cur], f)
    print("hello")
    output.clear()
    if (self.cur < len(self.to_annot) and inc == 1) or (self.cur > 0 and inc == -1):
      self.cur += inc
    self.run()

  def console(self, s):
    print(s)

  def prediction(self):
    if (self.model is not None) and (('entities' not in self.to_annot[self.cur]) or (len(self.to_annot[self.cur]['entities']) == 0)):
      doc = self.model(self.to_annot[self.cur]["text"])
      self.to_annot[self.cur]['entities'] = []
      for ent in doc.ents:
        self.to_annot[self.cur]['entities'].append([ent.start_char, ent.end_char, ent.label_])
      output.clear()
      self.run()


In [0]:
w = AnnotWidget(data, start=0, model=None)

In [0]:
w.run()

In [4]:
data = {x['file'] : x for x in data}
data_label = {x['file'] : x for x in data_label}
data_no_label = {x['file'] : x for x in data_no_label}
for x in w.to_annot:
  if 'entities' in x and len(x['entities']) != 0:
    data[x['file']] = x
    data_label[x['file']] = x
    if x['file'] in data_no_label:
      del data_no_label[x['file']]
data = [v for k, v in sorted(data.items(), key=lambda t: t[0])]
data_label = [v for k, v in sorted(data_label.items(), key=lambda t: t[0])]
data_no_label = [v for k, v in sorted(data_no_label.items(), key=lambda t: t[0])]
print(len(data), "texts with", len(data_label), "labeled and", len(data_no_label), 'to label')

NameError: ignored

## Download Json

In [39]:
from google.colab import files

!zip dataset.zip ./data/*.json
files.download('dataset.zip') 

  adding: data/0070C022_220C0700.json (deflated 66%)
  adding: data/0340C022_220C0430.json (deflated 59%)
  adding: data/0510C022_220C0150.json (deflated 51%)
  adding: data/0540C022_220C0450.json (deflated 62%)
  adding: data/0620C022_220C0260.json (deflated 50%)
  adding: data/0810C022_220C0180.json (deflated 69%)
  adding: data/0820C022_220C0280.json (deflated 60%)
  adding: data/0900C022_220C0090.json (deflated 74%)
  adding: data/0940C022_220C0490.json (deflated 56%)
  adding: data/0960C022_220C0690.json (deflated 69%)
  adding: data/0980C022_220C0890.json (deflated 54%)
  adding: data/1040C022_220C0401.json (deflated 61%)
  adding: data/1080C022_220C0801.json (deflated 51%)
  adding: data/1090C022_220C0901.json (deflated 66%)
  adding: data/1200C022_220C0021.json (deflated 58%)
  adding: data/1270C022_220C0721.json (deflated 64%)
  adding: data/1360C022_220C0631.json (deflated 65%)
  adding: data/1380C022_220C0831.json (deflated 55%)
  adding: data/1430C022_220C0341.json (deflate

## Train NER

In [40]:
split = 0.33
train_data = []
for x in data_label:
  entities = [tuple(ent) for ent in  x["entities"]]
  train_data.append((x['text'], {'entities' : entities}))
labels = ['ISSUER', 'FIRM', 'NUMBER', 'PERCENT', 'TYPE', 'DIRECTION', 'DATE', 'FILIALE', 'INSTRUMENT']
split = int(split * len(train_data))
test_data = train_data[:split]
train_data = train_data[split:]
print(len(test_data), len(train_data))

38 79


In [0]:
import random
import spacy
from spacy.util import minibatch, compounding
import fr_core_news_md

n_iter=40
nlp = fr_core_news_md.load()
ner = nlp.get_pipe("ner")
for label in labels:
  ner.add_label(label)
optimizer = nlp.resume_training()
pipe_exceptions = ["ner"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*other_pipes):  # only train NER
  sizes = compounding(1.0, 4.0, 1.001)
  # batch up the examples using spaCy's minibatch
  for itn in range(n_iter):
    random.shuffle(train_data)
    batches = minibatch(train_data, size=sizes)
    losses = {}
    for batch in batches:
      texts, annotations = zip(*batch)
      nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
    print("Losses", losses)
  score = nlp.evaluate(test_data)

Losses {'ner': 20281.23872630823}
Losses {'ner': 18359.395397767425}
Losses {'ner': 18034.401956422953}
Losses {'ner': 17661.076287855394}
Losses {'ner': 17754.452405526303}
Losses {'ner': 17263.59386597504}
Losses {'ner': 17120.939673734232}
Losses {'ner': 17112.690230745124}
Losses {'ner': 17258.234536451288}
Losses {'ner': 17194.213365793228}
Losses {'ner': 16996.494131974876}
Losses {'ner': 17171.376155279577}
Losses {'ner': 16872.84184090793}
Losses {'ner': 16799.044024169445}
Losses {'ner': 17211.805298149586}
Losses {'ner': 16804.13332286477}
Losses {'ner': 16984.96086921543}
Losses {'ner': 16796.667815968394}
Losses {'ner': 16621.641098976135}
Losses {'ner': 17223.622693538666}
Losses {'ner': 16959.608881918248}
Losses {'ner': 16742.974809485866}
Losses {'ner': 16795.66036939621}
Losses {'ner': 16653.796668052673}
Losses {'ner': 16932.72894887626}
Losses {'ner': 16924.9821113348}
Losses {'ner': 16736.068982839584}
Losses {'ner': 17094.829356193542}
Losses {'ner': 16693.39794480

In [0]:
score.scores

{'ents_f': 89.84771573604061,
 'ents_p': 88.05970149253731,
 'ents_per_type': {'DATE': {'f': 98.4126984126984, 'p': 96.875, 'r': 100.0},
  'DIRECTION': {'f': 100.0, 'p': 100.0, 'r': 100.0},
  'FILIALE': {'f': 85.71428571428571,
   'p': 85.71428571428571,
   'r': 85.71428571428571},
  'FIRM': {'f': 50.0, 'p': 41.17647058823529, 'r': 63.63636363636363},
  'INSTRUMENT': {'f': 66.66666666666666,
   'p': 66.66666666666666,
   'r': 66.66666666666666},
  'ISSUER': {'f': 89.1566265060241,
   'p': 84.0909090909091,
   'r': 94.87179487179486},
  'NUMBER': {'f': 85.71428571428572, 'p': 90.0, 'r': 81.81818181818183},
  'PERCENT': {'f': 97.67441860465117, 'p': 100.0, 'r': 95.45454545454545},
  'TYPE': {'f': 94.62365591397848,
   'p': 95.65217391304348,
   'r': 93.61702127659575}},
 'ents_r': 91.70984455958549,
 'las': 0.0,
 'las_per_type': {'': {'f': 0.0, 'p': 0.0, 'r': 0.0}},
 'tags_acc': 0.0,
 'textcat_score': 0.0,
 'textcats_per_cat': {},
 'token_acc': 100.0,
 'uas': 0.0}

In [0]:
from spacy import displacy
doc = nlp(data[0]['text'])

displacy.render(doc, style='ent', jupyter=True)

In [0]:
for t in test_data:
  doc = nlp(t[0])
  label_set = set([(e_s, e_e, e_t) for e_s, e_e, e_t in t[1]['entities']])
  pred_set = set([(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents])
  #print(label_set)
  print(pred_set - label_set)
  print(label_set - pred_set)


{(279, 290, 'DATE'), (1121, 1127, 'ISSUER'), (590, 609, 'ISSUER')}
{(590, 591, 'ISSUER'), (1121, 1140, 'ISSUER'), (591, 609, 'ISSUER'), (1928, 1947, 'ISSUER'), (2477, 2501, 'INSTRUMENT'), (279, 289, 'DATE')}
{(1109, 1112, 'TYPE'), (511, 516, 'ISSUER'), (1608, 1618, 'INSTRUMENT'), (804, 814, 'INSTRUMENT'), (1113, 1118, 'ISSUER'), (1103, 1108, 'NUMBER'), (589, 595, 'ISSUER')}
{(589, 594, 'ISSUER'), (511, 516, 'FIRM')}
{(171, 190, 'ISSUER'), (552, 562, 'INSTRUMENT')}
{(171, 191, 'ISSUER')}
{(697, 718, 'ISSUER'), (904, 925, 'ISSUER'), (216, 237, 'ISSUER'), (1878, 1892, 'ISSUER'), (1338, 1352, 'ISSUER'), (611, 625, 'ISSUER'), (928, 942, 'ISSUER'), (587, 608, 'ISSUER'), (1314, 1335, 'ISSUER'), (1642, 1663, 'ISSUER'), (297, 311, 'DATE'), (1987, 2008, 'ISSUER'), (1854, 1875, 'ISSUER'), (1666, 1680, 'ISSUER'), (1546, 1567, 'ISSUER'), (240, 254, 'ISSUER')}
{(1987, 2025, 'ISSUER'), (697, 735, 'ISSUER'), (1854, 1892, 'ISSUER'), (587, 625, 'ISSUER'), (216, 254, 'ISSUER'), (1642, 1680, 'ISSUER'), (1

In [0]:
import pandas as pd

def link_quantities(text, entities, dist_max=15):
  linked = []
  quantities = []
  pos_prev = None
  quant_type = None
  prev_type = None
  for ent_start, ent_end, ent_type in entities:
    ent_text = text[ent_start:ent_end]
    if ent_type in ['NUMBER', 'PERCENT']:
      if (prev_type == ent_type and ent_start - prev_pos < dist_max):
        quantities.append(ent_text)
        quant_type = ent_type
      else:
        quantities = [ent_text]
        quant_type = ent_type
    elif ent_type == 'TYPE':
      if len(quantities) != 0 and (prev_type in ['NUMBER', 'PERCENT', 'TYPE']) and ent_start - prev_pos < dist_max:
        linked.append((quantities, ent_text, quant_type))
      else:
        quantities = []
    prev_type = ent_type
    prev_pos = ent_end
  return pd.DataFrame.from_records(linked, columns=['quants', 'unit', 'type'])

def get_quantities(df, idx):
  if idx >= len(df):
    return None
  s = df.iloc[idx]
  ret = []
  for q in s['quants']:
    if s['type'] == 'PERCENT':
      if '/' in q:
        num, den = q.split('/')
        ret.append(float(num)/float(den)*100.0)
      else:
        ret.append(float(q.replace(',', '.').replace('%', '')))
    else:
      if 'aucun' in q:
        ret.append(0)
      else:
        ret.append(int(q.replace(' ', '')))
  return ret



def exctract_info(text, entities):
  df = pd.DataFrame.from_records([{'text':text[s:e], 'start':s, 'end':e, 'type':t} for s, e, t in entities])
  ret = {}
  ret['Issuer'] = df[df['type'] == 'ISSUER']['text'].value_counts().index[0]
  ret['Firms'] = df[df['type'] == 'FIRM']['text'].value_counts(sort=False).index.to_list()
  dir_idx = df[df['type'] == 'DIRECTION'].index[0]
  ret['Direction'] = df[df['type'] == 'DIRECTION']['text'].iloc[0]
  ret['Date'] = df[(df['type'] == 'DATE') & (df.index > dir_idx)]['text'].iloc[0]
  ret['Filiales'] = df[df['type'] == 'FILIALE']['text'].value_counts(sort=False).index.to_list()
  ret['Instruments'] = df[df['type'] == 'INSTRUMENT']['text'].value_counts(sort=False).index.to_list()
  df_q = link_quantities(text, entities)
  cap = df_q[(df_q['type'] == 'PERCENT') & df_q['unit'].str.contains('capital')]
  vote = df_q[(df_q['type'] == 'PERCENT') & df_q['unit'].str.contains('vote')]
  ret['Action'] = get_quantities(df_q[(df_q['type'] == 'NUMBER') & df_q['unit'].str.contains('action')], 0)
  ret['ThreasholdCapital'] = get_quantities(cap, 0)
  ret['Capital'] = get_quantities(cap, 1)
  ret['ThreasholdVote'] = get_quantities(vote, 0)
  ret['Vote'] = get_quantities(vote, 1)

  print(ret)


for d in test_data:
  exctract_info(d[0], d[1]['entities'])

{'Issuer': 'ALTRAN TECHNOLOGIES', 'Firms': ['Bank of America Corporation'], 'Direction': 'en hausse', 'Date': '17 février 2020', 'Filiales': ['Merrill Lynch International', 'BofA Securities Europe SA'], 'Instruments': ['cash-settled equity swap'], 'Action': [14377895], 'ThreasholdCapital': [5.0], 'Capital': [5.59], 'ThreasholdVote': [5.0], 'Vote': [5.58]}
{'Issuer': 'VALEO', 'Firms': ['VALEO', 'BlackRock, Inc.'], 'Direction': 'en baisse', 'Date': '29 janvier 2020', 'Filiales': [], 'Instruments': ['contracts for differences'], 'Action': [11878005], 'ThreasholdCapital': [5.0], 'Capital': [4.93], 'ThreasholdVote': [5.0], 'Vote': [4.81]}
{'Issuer': 'ALTRAN TECHNOLOGIES', 'Firms': ['Société Générale'], 'Direction': 'en baisse', 'Date': '9 janvier 2020', 'Filiales': [], 'Instruments': [], 'Action': None, 'ThreasholdCapital': [5.0], 'Capital': None, 'ThreasholdVote': [5.0], 'Vote': None}
{'Issuer': 'INTERNATIONAL FLAVORS & FRAGRANCES INC', 'Firms': ['BlackRock Inc.'], 'Direction': 'en hausse'

In [0]:
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.label_, chunk.root.text)

 220C0426-FR0000121964-FS0115             220C0426 FR0000121964-FS0115 31 janvier 2020    NP 220C0426-FR0000121964-FS0115
de franchissement de seuils NP franchissement
article L. 233 NP article
7 du code de commerce)    KLEPIERRE  (Euronext Paris)    Par courrier NP code
Bankplassen 2 NP Bankplassen
, Sentrum NP Sentrum
les seuils de 5% du capital et des droits de vote de la société KLEPIERRE et détenir 15 072 929 actions KLEPIERRE2 NP seuils
de droits de vote NP droits
soit 4,98% du capital et des droits de vote de cette société3 NP %
Ce franchissement de seuils résulte d’une restitution d’actions KLEPIERRE à titre de collatéral.   NP franchissement
_ NP _
_ NP _
_ NP _
_ NP _
_ NP _
_                                                             NP _
1 La Banque Centrale de Norvège. NP Banque
Dont NP Dont
126 actions détenues à titre de collatéral NP actions
Sur la base d'un capital composé de 302 664 095 actions représentant autant de droits de vote, en application du 2ème alinéa de l

## Hugging face

In [44]:
!pip uninstall -y transformers

Uninstalling transformers-2.8.0:
  Successfully uninstalled transformers-2.8.0


In [45]:
!pip install git+https://github.com/huggingface/transformers
!pip install seqeval

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_hzbfb9p
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-_hzbfb9p
  Created wheel for transformers: filename=transformers-2.8.0-cp36-none-any.whl size=585132 sha256=1c5f834e91dc5bcec8b91d8327cef2e31c375f2d798bdb5a4fadfa1facf0c045
  Stored in directory: /tmp/pip-ephem-wheel-cache-lv43mti_/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers


In [0]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# Named entity recognition pipeline, passing in a specific model and tokenizer
model = AutoModelForTokenClassification.from_pretrained("/content/hugging-res/checkpoint-70")
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
ner = pipeline('ner', model=model, tokenizer=tokenizer)

In [189]:
for t in train_data:
  a = ner(t[0])
  if len(a) != 0:
    print(a)

[{'word': 'CG', 'score': 0.9546008110046387, 'entity': 'ISSUER'}, {'word': 'Co', 'score': 0.33023521304130554, 'entity': 'ISSUER'}, {'word': '5', 'score': 0.6447892785072327, 'entity': 'PERCENT'}, {'word': 'société', 'score': 0.6966977715492249, 'entity': 'ISSUER'}, {'word': 'CG', 'score': 0.9425562024116516, 'entity': 'ISSUER'}, {'word': '##G', 'score': 0.5902494788169861, 'entity': 'ISSUER'}, {'word': 'CG', 'score': 0.9556360244750977, 'entity': 'ISSUER'}, {'word': '##G', 'score': 0.5657050609588623, 'entity': 'ISSUER'}, {'word': '5', 'score': 0.5227532982826233, 'entity': 'PERCENT'}, {'word': ',', 'score': 0.46169084310531616, 'entity': 'PERCENT'}, {'word': '03', 'score': 0.7963652610778809, 'entity': 'PERCENT'}, {'word': '%', 'score': 0.45184648036956787, 'entity': 'PERCENT'}, {'word': 'actions', 'score': 0.4320594370365143, 'entity': 'ISSUER'}, {'word': 'CG', 'score': 0.9460681080818176, 'entity': 'ISSUER'}, {'word': '##G', 'score': 0.6429470777511597, 'entity': 'ISSUER'}, {'word'

In [41]:
for s in split_to_sequences(train_data):
  for ent_s, ent_e, ent_t in s[1]['entities']:
    print(s[0][ent_s:ent_e], ent_t)

21 février 2020 DATE
CGG ISSUER
21 février 2020 DATE
Morgan Stanley & Co International plc FIRM
en hausse DIRECTION
17 février 2020 DATE
5% PERCENT
capital TYPE
droits de vote TYPE
CGG ISSUER
35 732 995 NUMBER
actions TYPE
CGG ISSUER
5,03% PERCENT
capital TYPE
droits de vote TYPE
CGG ISSUER
Morgan Stanley Corp. FILIALE
35 742 697 NUMBER
actions TYPE
CGG ISSUER
5,03% PERCENT
capital TYPE
droits de vote TYPE
Morgan Stanley & Co. LLC FILIALE
9 702 NUMBER
actions TYPE
CGG ISSUER
CGG ISSUER
Morgan Stanley & Co. International plc FIRM
31 223 NUMBER
actions TYPE
CGG ISSUER
right to recall INSTRUMENT
CGG ISSUER
Morgan Stanley & Co. FIRM
International plc FIRM
171 500 NUMBER
actions TYPE
CGG ISSUER
equity swap INSTRUMENT
16 mars 2020 DATE
Morgan Stanley Corp. FIRM
709 960 012 NUMBER
actions TYPE
710 098 578 NUMBER
droits de vote TYPE
10 mars 2020 DATE
CARREFOUR ISSUER
10 mars 2020 DATE
JP Morgan Chase & Co. FIRM
en baisse DIRECTION
6 mars 2020 DATE
5% PERCENT
capital TYPE
droits de vote TYPE
CA

In [196]:
import torch
sequence = train_data[0][0][:1000]
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt")

outputs = model(inputs)[0]
predictions = torch.argmax(outputs, dim=2)

print([(token, labels[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())])

[('[CLS]', 'O'), ('220', 'O'), ('##C', 'O'), ('##0', 'O'), ('##71', 'O'), ('##3', 'O'), ('-', 'O'), ('FR', 'O'), ('##00', 'O'), ('##13', 'O'), ('##18', 'O'), ('##18', 'O'), ('##64', 'O'), ('-', 'O'), ('FS', 'O'), ('##01', 'O'), ('##8', 'O'), ('##8', 'O'), ('220', 'O'), ('##C', 'O'), ('##0', 'O'), ('##71', 'O'), ('##3', 'O'), ('FR', 'O'), ('##00', 'O'), ('##13', 'O'), ('##18', 'O'), ('##18', 'O'), ('##64', 'O'), ('-', 'O'), ('FS', 'O'), ('##01', 'O'), ('##8', 'O'), ('##8', 'O'), ('21', 'O'), ('février', 'O'), ('2020', 'O'), ('D', 'O'), ('##éc', 'O'), ('##lara', 'O'), ('##tion', 'O'), ('de', 'O'), ('franc', 'O'), ('##his', 'O'), ('##sement', 'O'), ('de', 'O'), ('seu', 'O'), ('##ils', 'O'), ('(', 'O'), ('article', 'O'), ('L', 'O'), ('.', 'O'), ('233', 'O'), ('-', 'O'), ('7', 'O'), ('du', 'O'), ('code', 'O'), ('de', 'O'), ('commerce', 'O'), (')', 'O'), ('CG', 'ISSUER'), ('##G', 'ISSUER'), ('(', 'O'), ('Euro', 'O'), ('##ne', 'O'), ('##xt', 'O'), ('Paris', 'O'), (')', 'O'), ('Par', 'O'), ('c

In [48]:
! mkdir hugging-data
! mkdir hugging-res
! git clone https://github.com/huggingface/transformers

mkdir: cannot create directory ‘hugging-data’: File exists
mkdir: cannot create directory ‘hugging-res’: File exists
Cloning into 'transformers'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 24611 (delta 39), reused 22 (delta 8), pack-reused 24536
Receiving objects: 100% (24611/24611), 15.00 MiB | 25.55 MiB/s, done.
Resolving deltas: 100% (17270/17270), done.


In [0]:
def split_to_sequences(data, sep='  '):
  sequences = []
  for d in data:
    offset = 0
    start_ent = 0
    ent_idx = 0
    for seq in d[0].split(sep):
      if len(seq) != 0:
        entities = []
        next_offset = offset + len(seq) + len(sep)
        while ent_idx < len(d[1]['entities']):
          ent_s, ent_e, ent_t = d[1]['entities'][ent_idx]
          if ent_s < next_offset:
            entities.append((max(ent_s - offset, 0), min(ent_e - offset, len(seq)), ent_t))
            if ent_e < next_offset:
              ent_idx += 1
            else:
              break
          else:
            break
        sequences.append((seq, {'entities' : entities}))
        offset = next_offset
      else:
        offset += len(sep)
  return sequences

In [0]:
# camebert split tok '▁'
def transform_label(data, tokenizer, split_tok='##', unk_tok='[UNK]'):
  X = []
  Y = []
  for text, labels in data:
    offset = 0
    idx_ent = 0
    tokens = tokenizer.tokenize(text)
    y = []
    for tok in tokens:
      if idx_ent >= len(labels['entities']):
        y = y + ['O']*(len(tokens) - len(y))
        break
      offset_tok = len(split_tok) if tok.startswith(split_tok) else 0
      tok_s = None
      if tok == unk_tok:
        y.append('O')
        continue
      for offset_tok in range(offset_tok, len(tok)):
        while offset < len(text) and text[offset] != tok[offset_tok]:
          offset += 1
        if tok_s is None:
          tok_s = offset
        offset += 1
      tok_e = offset
      ent_s, ent_e, ent_t = labels['entities'][idx_ent]
      if tok_s is not None and tok_s >= ent_s:
        y.append(ent_t)
        if tok_e >= ent_e:
          idx_ent += 1
      else:
        y.append('O')
    X.append(tokens)
    Y.append(y)
  return X, Y

In [184]:
import pandas as pd
%load_ext google.colab.data_table

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
X, Y = transform_label(train_data, tokenizer)
df = pd.DataFrame()
df['X'] = X[0]
df['Y'] = Y[0]
df
#print(df)

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


,X,Y
0,220,O
1,##C,O
2,##0,O
3,##71,O
4,##3,O
...,...,...
703,un,O
704,delta,O
705,de,O
706,1,O


In [0]:
def labels_to_file(filename, labels):
  with open(filename, 'w+') as f:
    for l in labels:
      f.write(l + '\n')
  
def data_to_file(filename, X, Y):
  with open(filename, 'w+') as f:
    for x, y in zip(X, Y):
      for tex, lab in zip(x, y):
        f.write(tex + ' ' + lab + '\n')
      f.write('\n')

In [56]:
test_seq = split_to_sequences(test_data)
for t in test_seq:
  for ent_s, ent_e, ent_t in t[1]['entities']:
    print(t[0][ent_s:ent_e], ent_t)

21 février 2020 DATE
ALTRAN TECHNOLOGIES ISSUER
20 février 2020 DATE
21 février DATE
Bank of America Corporation FIRM
en hausse DIRECTION
17 février 2020 DATE
5% PERCENT
capital TYPE
droits de vote TYPE
A ISSUER
LTRAN TECHNOLOGIES ISSUER
14 377 895 NUMBER
actions TYPE
ALTRAN TECHNOLOGIES ISSUER
5,59% PERCENT
capital TYPE
5,58% PERCENT
droits de vote TYPE
ALTRAN TECHNOLOGIES ISSUER
ALTRAN TECHNOLOGIES ISSUER
Merrill Lynch International FILIALE
en hausse DIRECTION
Merrill Lynch International FILIALE
13 784 025 NUMBER
actions TYPE
ALTRAN TECHNOLOGIES ISSUER
Merrill Lynch International FILIALE
BofA Securities Europe SA FILIALE
274 482 NUMBER
actions TYPE
ALTRAN TECHNOLOGIES ISSUER
BofA Securities Europe SA FILIALE
Merrill Lynch International FILIALE
17 749 NUMBER
actions TYPE
ALTRAN TECHNOLOGIES ISSUER
cash-settled equity swap INSTRUMENT
26 octobre 2020 DATE
30 octobre 2020 DATE
257 021 105 NUMBER
actions TYPE
257 652 732 NUMBER
droits de vote TYPE
31 janvier 2020 DATE
VALEO ISSUER
31 janv

In [0]:
from transformers import AutoTokenizer

labels = ['ISSUER', 'FIRM', 'NUMBER', 'PERCENT', 'TYPE', 'DIRECTION', 'DATE', 'FILIALE', 'INSTRUMENT', 'O']
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
train_seq = split_to_sequences(train_data)
test_seq = split_to_sequences(test_data)
X_train, Y_train = transform_label(train_seq, tokenizer)
X_test, Y_test = transform_label(test_seq, tokenizer)
labels_to_file('hugging-data/labels.txt', labels)
data_to_file('hugging-data/train.txt', X_train, Y_train)
data_to_file('hugging-data/dev.txt', X_test, Y_test)


In [0]:
cmd = """
    python run_ner.py
    --model_name bert-base-multilingual-cased
    --output_dir /content/hugging-res
    --overwrite_output_dir
    --data_dir /content/hugging-data
    --labels /content/hugging-data/labels.txt
    --max_seq_length 256
    --per_gpu_train_batch_size 32
    --num_train_epochs 24
    --save_steps 70
    --logging_steps 1
    --do_train
    --do_eval
""".replace('\n', ' ')

In [53]:
%cd /content/transformers/examples/ner/
! {cmd}
%cd /content/

/content/transformers/examples/ner
2020-04-24 14:59:54.124269: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
04/24/2020 14:59:58 - INFO - transformers.training_args -   PyTorch: setting up devices
04/24/2020 14:59:58 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/24/2020 14:59:58 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/content/hugging-res', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, per_gpu_train_batch_size=32, per_gpu_eval_batch_size=8, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=24.0, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=1, save_steps=70, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_o

In [0]:
import re
l = []
seq = []
for t in train_data:
  sents = t[0].split('. ')
  for s in sents:
    l.append(len(s))
  seq.append(sents)

In [121]:
import statistics
max(l)

1249

In [124]:
seq[0]

[' 220C0713-FR0013181864-FS0188              220C0713 FR0013181864-FS0188 21 février 2020   Déclaration de franchissement de seuils (article L',
 '233-7 du code de commerce)    CGG  (Euronext Paris)   Par courrier reçu le 21 février 2020, la société Morgan Stanley & Co International plc1 (25 Cabot Square, Canary Wharf, Londres E14 4QA, Royaume Uni), a déclaré avoir franchi individuellement en hausse, le 17 février 2020, les seuils de 5% du capital et des droits de vote de la société CGG et détenir individuellement 35 732 995 actions CGG représentant autant de droits de vote, soit 5,03% du capital et des droits de vote de cette société2',
 ' Ce franchissement de seuils résulte d’une acquisition d’actions CGG hors marché',
 ' À cette occasion, la société Morgan Stanley Corp',
 '(c/o The Corporation Trust Company (DE), Corporation Trust Center, 1209 Orange Street, Wilmington, Delaware DE 19801, Etats-Unis) n’a franchi aucun seuil et a précisé détenir, indirectement, 35 742 697 actions CGG

In [0]:
!rm -r /content/hugging-res/checkpoint-8

In [173]:
%cd /content/

/content
